# Игра угадай число

*также смотри [технические данные о среде выполнения](#технические-данные-о-среде-выполнения)

## Предпосылки

Для завершения модуля "PYTHON-8. Инструменты для Data Science" ставится задача требующая навыков владения инструментами, изложенными в данном модуле. Для этого предоставляется код, созданный к этому моменту в рамках модуля.

Давайте проверим его работоспособность.

### Импортируем функции модуля

In [1]:
from game_v2 import score_game
from game_v2 import random_predict

Здесь мы делаем доступными функции модуля game_v2:
- *score_game* - функция, выявляющая через [среднее арифмитическое](https://ru.wikipedia.org/wiki/Среднее_арифметическое) за сколько попыток переданная ей функция поиска находит результат;
- *random_predict* - функция поиска заданного числа [методом научного тыка](https://ru.wikipedia.org/wiki/Метод_проб_и_ошибок) без запоминания истории, возвращающая количество потребовавшихся попыток.

### Проверим работоспособность *random_predict* несколько раз

In [7]:
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)

Ваш алгоритм угадывает число в среднем за:98 попыток
Ваш алгоритм угадывает число в среднем за:96 попыток
Ваш алгоритм угадывает число в среднем за:101 попыток
Ваш алгоритм угадывает число в среднем за:96 попыток
Ваш алгоритм угадывает число в среднем за:99 попыток


99

Как видим, количество требующихся для нахождения заданного числа попыток варьируется около 100.

## Постановка задачи

Взяв предоставленный код за отправную точку добиться следующего:
 - новая программа должна достигать результата меннее чем за 20 попыток;
 - код программы должен быть предоставлен через GitHub;
 - также, созданный для решения данной задачи код необходимо оформить соответствующим для GitHub образом;
 - поскольку код создаётся на языке Python он должен соответствовать PEP8;
 - как и любой другой код, необходимо, чтобы он был воспроизводим.

## Выбор алгоритма

Возьмём за основу [алоритм бинарного поиска](https://ru.wikipedia.org/wiki/Двоичный_поиск). Следуя его правилам поиск числа от 1 до 100 не должен превысить 7 шагов.

Так, например, если загадано число 1, то следуя правилу не оставлять ниже выбранного для попытки числа меньше вариантов чем выше, получим следующие шаги:
1. 100/2 = 50 => 51
2. 50/2 = 25 => 26
3. 25/2 = 12.5 => 13
4. 12/2 = 6 => 7
5. 6/2 = 3 => 4
6. 3/2 = 1.5 => 2
7. 1/2 = 0.5 => 1

## Организация кода

Для обратной совместимости оставим предоставленный код в оригинальном виде. А новый код оформим в файле [*result_game.py*](result_game.py).

### Функция поиска

Создадим функцию *binary_search*:

In [55]:
from math import log2

def binary_search(hidden_number=1):
    interval = (1, 101) # mathematially half-open left-closed interval - [1, 101)
    max_attempts_to_panic = int(1 + log2(len(range(*interval))))
    
    for attempt_index in range(max_attempts_to_panic):
        
        predict_number = int(sum(interval) / 2)
        
        if predict_number == hidden_number:
            return attempt_index+1

        if predict_number < hidden_number:
            interval = (predict_number+1, interval[1])
        elif hidden_number < predict_number:
            interval = (interval[0], predict_number)

    raise RuntimeError('Algorithm working not like expected')

И проверим её работоспособность уже имеющейся функцией *score_game*:

In [35]:
from game_v2 import score_game

score_game(binary_search)
score_game(binary_search)
score_game(binary_search)
score_game(binary_search)
score_game(binary_search)

Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток


5

Таким образом легко убедиться в значительном преимуществе функции *binary_search* перед изначальной *random_predict*.

Также, можно заметить, что функция *binary_search* использует самоограничение по количеству попыток. Это сделано для того для того, чтобы случайно внесённая (или не выявленная при создании функции) ошибка не приводила к бесконечному циклу.

Убедимся в полезности этого самоограничения.

In [56]:
binary_search(0)

RuntimeError: Algorithm working not like expected

Однако, если задать выходящее за диапазон число 101:

In [58]:
binary_search(101)

7

То мы увидим, что фунция отработала, но не ожиданным образом. А именно нашла число не входящее в диапазон.

Что очередной раз свидетельствует о необходимости проверки входных параметров. Доработаем функцию *binary_search*.

In [59]:
from math import log2

def binary_search(hidden_number=1):
    if not(1 <= hidden_number <= 100):
        raise ValueError('Expected value from 1 to 100')
    
    interval = (1, 101) # mathematially half-open left-closed interval - [1, 101)
    max_attempts_to_panic = int(1 + log2(len(range(*interval))))
    
    for attempt_index in range(max_attempts_to_panic):
        
        predict_number = int(sum(interval) / 2)
        
        if predict_number == hidden_number:
            return attempt_index+1

        if predict_number < hidden_number:
            interval = (predict_number+1, interval[1])
        elif hidden_number < predict_number:
            interval = (interval[0], predict_number)

    raise RuntimeError('Algorithm working not like expected')

Проверим снова:

In [60]:
binary_search(101)

ValueError: Expected value from 1 to 100

Теперь можно сказать, что функция работает как надо. И хотя необходимость проверки входных параметров на соответсвие типу никто не отменял, будем считать, что поставленная задача выполнена. Поскольку в реальных задачах проверка входных параметров истекает из логики построения кода и излишние проверки только замедлят выполнение кода поставленной задачи.

## Технические данные о среде выполнения

In [4]:
import sys
print(sys.version)

3.9.12 (main, Apr 18 2022, 22:40:46) 
[GCC 9.4.0]


In [10]:
#pip freeze
#pip freeze > requirements.txt
#pip install -r requirements.txt